Allen Institute resources for API access:
- http://help.brain-map.org/display/api/Allen+Brain+Atlas+API
- http://help.brain-map.org/display/api/Downloading+3-D+Expression+Grid+Data
- http://help.brain-map.org/display/api/Example+Queries+for+Experiment+Metadata
- http://help.brain-map.org/pages/viewpage.action?pageId=5308449

XML parsing:
- https://www.datacamp.com/community/tutorials/python-xml-elementtree
- https://www.kite.com/python/answers/how-to-download-a-csv-file-from-a-url-in-python


In [3]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
# url = "http://api.brain-map.org/api/v2/data/query.csv?criteria=model::SectionDataSet,rma::criteria,[failed$eqfalse],products[abbreviation$eq'Mouse'],treatments[name$eq'ISH'],genes,plane_of_section,rma::options,[tabular$eq'plane_of_sections.name+as+plane','genes.acronym+as+gene','data_sets.id+as+section_data_set_id'],[order$eq'plane_of_sections.name,genes.acronym,data_sets.id']&num_rows=50&start_row=0"
url = "http://api.brain-map.org/api/v2/data/query.xml?criteria=model::SectionDataSet,rma::criteria,[failed$eqfalse],products[abbreviation$eq'Mouse'],treatments[name$eq'ISH'],rma::include,genes,specimen(donor(age)),plane_of_section&num_rows=200&start_row=0"

In [3]:
#This XML query gets the 4345 experiments in the coronal data set
"http://api.brain-map.org/api/v2/data/SectionDataSet/query.xml?criteria=[failed$eqfalse],plane_of_section[name$eqcoronal],products[abbreviation$eqMouse],treatments[name$eqISH]"

#This one gets the 21733 experiments in the sagittal data set
"http://api.brain-map.org/api/v2/data/SectionDataSet/query.xml?criteria=[failed$eqfalse],plane_of_section[name$eqsagittal],products[abbreviation$eqMouse],treatments[name$eqISH]"

'http://api.brain-map.org/api/v2/data/SectionDataSet/query.xml?criteria=[failed$eqfalse],plane_of_section[name$eqsagittal],products[abbreviation$eqMouse],treatments[name$eqISH]'

In [ ]:
# abi_brainmap_base = "http://api.brain-map.org/api/v2/data"
# abi_brainmap_model = "/SectionDataSet"
# abi_brainmap_queryxml = "/query.xml?"
# abi_brainmap_criteria = {}

In [13]:
root = ET.fromstring(r.content)

In [10]:
root.tag

'Response'

In [14]:
root.attrib

{'success': 'true', 'start_row': '0', 'num_rows': '1', 'total_rows': '4345'}

In [15]:
abi_brainmap_query_init = "http://api.brain-map.org/api/v2/data/SectionDataSet/query.xml?criteria=[failed$eqfalse],plane_of_section[name$eqcoronal],products[abbreviation$eqMouse],treatments[name$eqISH]&start_row=0&num_rows=1"

r = requests.get(abi_brainmap_query_init)

root = ET.fromstring(r.content)
nfiles = int(root.attrib['total_rows'])

In [18]:
url = "http://api.brain-map.org/api/v2/data/SectionDataSet/query.csv?criteria=[failed$eqfalse],products[abbreviation$eqMouse],treatments[name$eqISH],plane_of_section[name$eqcoronal],genes&tabular=genes.acronym+as+gene,data_sets.id+as+ExperimentID,plane_of_sections.name+as+plane&start_row=4340&num_rows=10"


# %20rma::options,%20[tabular$eq%27plane_of_sections.name+as+plane%27,%27genes.acronym+as+gene%27,%27data_sets.id+as+section_data_set_id%27],%20[order$eq%27plane_of_sections.name,genes.acronym,data_sets.id%27]%20&num_rows=50&start_row=0"

In [4]:
import pandas as pd

In [19]:
pd.read_csv(url)

,gene,experimentid,plane
0,Cyp26b1,72081548,coronal
1,Kcng4,72081560,coronal
2,Glra1,72108824,coronal
3,Ace,72109489,coronal
4,Rnf13,72109423,coronal


In [27]:
abi_query_base = "http://api.brain-map.org/api/v2/data/SectionDataSet/query.csv?"+\
"criteria=[failed$eqfalse],products[abbreviation$eqMouse],treatments[name$eqISH],plane_of_section[name$eqcoronal],genes&"+\
"tabular=genes.acronym+as+gene,data_sets.id+as+ExperimentID,plane_of_sections.name+as+plane&"

# start_row=4340&num_rows=10"

num_rows = 100
start_row = 0
end_row = start_row + num_rows - 1
flag = 1

In [28]:
abi_query = abi_query_base+"start_row={}&num_rows={}".format(start_row, num_rows)

In [16]:
nfiles = 5

4345

In [17]:
while flag == 1:
    abi_query = abi_query_base+"start_row={}&num_rows={}".format(start_row, num_rows)
    
    

True

In [9]:
abi_query_base = "http://api.brain-map.org/api/v2/data/SectionDataSet/query.csv?"+\
"criteria=[failed$eqfalse],products[abbreviation$eqMouse],treatments[name$eqISH],plane_of_section[name$eqcoronal],genes&"+\
"tabular=genes.acronym+as+gene,data_sets.id+as+ExperimentID,plane_of_sections.name+as+plane&"

start_row = 0
num_rows = 'all'
abi_query = abi_query_base+"start_row={}&num_rows={}".format(start_row, num_rows)
abi_query
temp = pd.read_csv(abi_query)

'http://api.brain-map.org/api/v2/data/SectionDataSet/query.csv?criteria=[failed$eqfalse],products[abbreviation$eqMouse],treatments[name$eqISH],plane_of_section[name$eqcoronal],genes&tabular=genes.acronym+as+gene,data_sets.id+as+ExperimentID,plane_of_sections.name+as+plane&start_row=0&num_rows=all'

In [45]:
temp_id = temp.loc[0, 'experimentid']

In [51]:
url = 'http://api.brain-map.org/grid_data/download/{}'.format(temp_id)
url

'http://api.brain-map.org/grid_data/download/112646890'

In [52]:
r = requests.get(url)


In [54]:
outfile = 'Data/Expression/temp.zip'
with open(outfile, 'wb') as file:
    file.write(r.content)

In [5]:
temp = pd.read_csv("/projects/yyee/tools/Allen_brain/data/gene_data.csv")

In [6]:
temp.head()

,success,id,start_row,num_rows,total_rows,msg.blue_channel,msg.delegate,msg.expression,msg.failed,msg.failed_facet,...,msg.genes.homologene_id,msg.genes.id,msg.genes.legacy_ensembl_gene_id,msg.genes.name,msg.genes.organism_id,msg.genes.original_name,msg.genes.original_symbol,msg.genes.reference_genome_id,msg.genes.sphinx_id,msg.genes.version_status
0,True,0,0,28848,28850,NaN,False,True,False,734881840,...,12986.0,88552,NaN,"SLIT and NTRK-like family, member 6",2,"SLIT and NTRK-like family, member 6",Slitrk6,NaN,100744,no change
1,True,0,0,28848,28850,NaN,False,True,False,734881840,...,NaN,130779,NaN,LOC434199,2,LOC434199,LOC434199,NaN,11669,deleted
2,True,0,0,28848,28850,NaN,False,True,False,734881840,...,1646.0,15970,NaN,interleukin 7 receptor,2,interleukin 7 receptor,Il7r,NaN,92339,no change
3,True,0,0,28848,28850,NaN,False,True,False,734881840,...,NaN,129772,NaN,hypothetical gene supported by AK082053,2,hypothetical gene supported by AK082053,LOC433187,NaN,160096,not current
4,True,0,0,28848,28850,NaN,False,True,False,734881840,...,8040.0,23765,NaN,"ribonuclease L (2', 5'-oligoisoadenylate synth...",2,"ribonuclease L (2', 5'-oligoisoadenylate synth...",Rnasel,NaN,343660,no change


In [7]:
temp.columns

Index(['success', 'id', 'start_row', 'num_rows', 'total_rows',
       'msg.blue_channel', 'msg.delegate', 'msg.expression', 'msg.failed',
       'msg.failed_facet', 'msg.green_channel', 'msg.id', 'msg.name',
       'msg.plane_of_section_id', 'msg.qc_date', 'msg.red_channel',
       'msg.reference_space_id', 'msg.rnaseq_design_id',
       'msg.section_thickness', 'msg.specimen_id', 'msg.sphinx_id',
       'msg.storage_directory', 'msg.weight', 'msg.genes.acronym',
       'msg.genes.alias_tags', 'msg.genes.chromosome_id',
       'msg.genes.ensembl_id', 'msg.genes.entrez_id',
       'msg.genes.genomic_reference_update_id', 'msg.genes.homologene_id',
       'msg.genes.id', 'msg.genes.legacy_ensembl_gene_id', 'msg.genes.name',
       'msg.genes.organism_id', 'msg.genes.original_name',
       'msg.genes.original_symbol', 'msg.genes.reference_genome_id',
       'msg.genes.sphinx_id', 'msg.genes.version_status'],
      dtype='object')

In [47]:
abi_query = "http://api.brain-map.org/api/v2/data/SectionDataSet/query.csv?"+\
"criteria=[failed$eqfalse],plane_of_section[name$eqcoronal],products[abbreviation$eqMouse],treatments[name$eqISH]&"+\
"tabular=data_sets.id+as+experiment_id,data_sets.section_thickness,data_sets.specimen_id,plane_of_sections.name+as+plane,genes.acronym"


# "start_row=0&num_rows=all"


# "tabular=genes.acronym+as+gene,data_sets.id+as+ExperimentID,plane_of_sections.name+as+plane&"

This isn't working with the genes table for some reason

In [48]:
temp = pd.read_csv(abi_query)
temp

,Data Access error in query: model::SectionDataSet,rma::criteria,[failed$eqfalse],plane_of_section[name$eqcoronal],products[abbreviation$eqMouse],treatments[name$eqISH],rma::options[tabular$eq'data_sets.id as experiment_id,data_sets.section_thickness,data_sets.specimen_id,plane_of_sections.name as plane,genes.acronym']
